# [CS 6320 Project 3: Local Feature Matching]
<img src="figures/eval.jpg" alt="drawing" width="600" title="eval"/>


## Brief
    Hand-in: through Canvas
    Required files: <your_uid>.zip.(Please begin with 'u' for your uid)
                    <your_uid>_proj3.pdf

## Overview

The goal of this assignment is to create a local feature matching algorithm using techniques described in Szeliski chapter 4.1. The pipeline we suggest is based on a simplified version of the famous SIFT pipeline. However, we will not be implementing this in the classical fashion. We will be implementing it as though it were part of a neural network. The matching pipeline is intended to work for instance-level matching – multiple views of the same physical scene.

This project is intended to further familiarize you with Python, PyTorch, and local feature matching. Once again, you may find these resources helpful. Python: [here](https://docs.python.org/3/tutorial/). PyTorch: [here](https://pytorch.org/tutorials/).

### Setup

   0. Unzip proj3_6320.zip and go to proj3_6320 directory.
      - You can run `unzip proj3_6320.zip && cd proj3_6320` in your terminal.
   1. Install [Miniconda](https://docs.conda.io/en/latest/miniconda.html). It doesn’t matter whether you use Python 2 or 3 because we will create our own environment that uses 3 anyways.
   2. Create a conda environment using the appropriate command. On Windows, open the installed “Conda prompt” to run the command. On MacOS and Linux, you can just use a terminal window to run the command, Modify the command based on your OS (linux, mac, or win): `conda env create -f proj3_env_<OS>.yml`.
    - NOTE that proj3_env_<OS>.yml is inside the project folder.
   3. This should create an environment named ‘proj3’. Activate it using the Windows command, activate proj3 or the MacOS / Linux command, source activate proj3
   4. Install the project package, by running `pip install -e .` inside the repo folder.
   5. Run the notebook using `jupyter notebook` under *proj3_6320* directory.
   6. Ensure that all sanity checks are passing by running pytest tests inside the repo folder.
   7. Generate the zip folder for the code portion of your submission once you’ve finished the project using 
    
        `python zip_submission.py --uid <your_uid>` 


<!-- ## Library Functions 
Do not use any library functions to implement Hough Transform. -->


### Writeup
For this project, you need to run all your cells and then convert the notebook into pdf. A conversion way is to use the tool provided by Jupyter. Click the items on the menu of this website page:

`File -> Download as -> PDF via LaTeX(.pdf)`
    
You can refer [here](1) in case you encounter any problem.
    
[1]: https://stackoverflow.com/questions/29156653/ipython-jupyter-problems-saving-notebook-as-pdf
    
Your code, results, visualization, and discussion will be used for the grading. You will be deducted points if the results are not shown in this notebook. Do not change the order of the cells. You can add cells in need. You can copy a cell and run it seperately if you need to run a cell multiple times and thus every result is displayed in the cell.


### Rubric
* +40 pts: HarrisNet implementation in HarrisNet.py    
* +40 pts: SIFTNet implementation in SIFTNet.py
* +10 pts: Feature matching implementation in student_feature_matching.py
* +5 pts: pdf report

Distribution of the points in a Question is separately mentioned for each sub-task

* -5*n pts: Lose 5 points for every time you do not follow the instructions for the hand-in format.
    
### Submission Format

This is very important as you will lose 5 points for every time you do not follow the instructions. You will attach two items in your submission on Canvas:

1. `<your_uid>`.zip containing:
    `<your_gt_username>`.zip via Canvas containing:
    - proj3_code/ - directory containing all your code for this assignment (including HarrisNet.py, SIFTNet.py, !)
    - additional_data/ - (optional) if you use any data other than the images we provide you, please include them here
    - README.txt - (optional) if you implement any new functions other than the ones we define in the skeleton code (e.g. any extra credit implementations), please describe what you did and how we can run the code. We will not award any extra credit if we can’t run your code and verify the results.

2. `<your_gt_username>`_proj3.pdf via Gradescope - your report

Do not install any additional packages inside the conda environment. The TAs will use the same environment as defined in the config files we provide you, so anything that’s not in there by default will probably cause your code to break during grading. Do not use absolute paths in your code or your code will break. Use relative paths like the starter code already does. Failure to follow any of these instructions will lead to point deductions. Create the zip file using python zip_submission.py --uid `<your_uid>` (it will zip up the appropriate directories/files for you!)

## This iPython notebook:  
    (1) Loads and resizes images  
    (2) Finds interest points in those images                 **(you code this)**  
    (3) Describes each interest point with a local feature    **(you code this)**  
    (4) Finds matching features                               **(you code this)**  
    (5) Visualizes the matches  
    (6) Evaluates the matches based on ground truth correspondences  
### Data
We provide you with 3 pairs of pictures of the Notre Dame, Mt. Rushmore, and the Episcopal Palace(which we refer to as Gaudi). Each image in a pair is of the same object but from a slightly different viewpoint, and was taken under differing conditions. These images will be run through the entire local feature matching pipeline, first extracting interest points, then extracting features, and finally feature matching to get correspondences across images. The image at the top of the page is what the final evaluation looks like. **Interest points are matched across images and correct ones (according to an annotated ground truth) are marked with <span style="color:green">green lines</span> and incorrect with <span style="color:red">red</span>**. You are also free to test images of your own, you will just have to annotate them with a script we provide you with in the annotate_correspondences folder.

**<span style="color:red">Your accuracy on the Notre Dame image pair must be at least 80% for the 100 most confident matches to receive full credit!</span>**

**Potentially useful NumPy Python library) or pytorch functions**: There are more details for these in each specific function header.

**Forbidden functions** (you can use these for testing, but not in your final code): anything that takes care of any of the main functions you’ve been asked to implement. If it feels like you’re sidestepping the work, then it’s probably not allowed. Ask the TAs if you have any doubts.


**Additional_data**: If you want to test on your own data/images, please put them in the subdirectory 'proj3_6320/additional_data', which will be included in the zip file for the submission. 

**IMPORTANT**: **<span style="color:red">PLEASE START THIS PROJECT EARLY</span> as you might need much effort on the pytorch functions. 

### Testing
We have provided a set of tests for you to evaluate your implementation. We have included tests inside proj3.ipynb so you can check your progress as you implement each section. When you’re done with the entire project, you can call additional tests by running pytest unit_tests inside the root directory of the project. **Your grade on the coding portion of the project will be further evaluated with a set of tests not provided to you**.

## Set up

In [1]:
import sys
sys.path.append('..')

In [2]:
%matplotlib inline
%matplotlib notebook
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import numpy as np

from proj3_code.utils import load_image, PIL_resize, rgb2gray
from IPython.core.debugger import set_trace
import torch
import torchvision
import torchvision.transforms as transforms

# Notre Dame
image1 = load_image('../data/1a_notredame.jpg')
image2 = load_image('../data/1b_notredame.jpg')
eval_file = '../ground_truth/notredame.pkl'

# # Mount Rushmore -- this pair is relatively easy (still harder than Notre Dame, though)
# image1 = load_image('../data/2a_rushmore.jpg')
# image2 = load_image('../data/2b_rushmore.jpg')
# eval_file = '../ground_truth/rushmore.pkl'

# # Episcopal Gaudi -- This pair is relatively difficult
# image1 = load_image('../data/3a_gaudi.jpg')
# image2 = load_image('../data/3b_gaudi.jpg')
# eval_file = '../ground_truth/gaudi.pkl'

scale_factor = 0.5
image1 = PIL_resize(image1, (int(image1.shape[1]*scale_factor), int(image1.shape[0]*scale_factor)))
image2 = PIL_resize(image2, (int(image2.shape[1]*scale_factor), int(image2.shape[0]*scale_factor)))

image1_bw = rgb2gray(image1)
image2_bw = rgb2gray(image2)
plt.figure(figsize=(4,4)); plt.imshow((image1_bw*255).astype(np.uint8), cmap='gray');
plt.figure(figsize=(4,4)); plt.imshow((image2_bw*255).astype(np.uint8), cmap='gray');

#convert images to tensor
tensor_type = torch.FloatTensor
torch.set_default_tensor_type(tensor_type)
to_tensor = transforms.ToTensor()

image_input1 = to_tensor(image1_bw).unsqueeze(0)
image_input2 = to_tensor(image2_bw).unsqueeze(0)
print('shape of image1_bw:', image_input1.shape)
print('shape of image2_bw:', image_input2.shape)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

shape of image1_bw: torch.Size([1, 1, 1024, 768])
shape of image2_bw: torch.Size([1, 1, 1016, 762])



# Part 1: Harris Corner Detector 
**NOTE**: Before starting this project, please look at [here](https://cs231n.github.io/convolutional-networks/) and [here]( http://cs231n.stanford.edu/slides/2018/cs231n_2018_lecture05.pdf) for better understanding of CNN in case you are confused about the convolutional operation.
You'd better also understand [group](https://discuss.pytorch.org/t/conv2d-certain-values-for-groups-and-out-channels-dont-work/14228/2) parametes well for nn.Conv2d(). 
## Find distinctive points in each image (Szeliski 4.1.1)

<img src="figures/HarrisNet.png" alt="drawing" width="800" title="Overview of HarrisNet pipeline"/>

The original Harris corner detector is described in the lecture materials(see S04-Features-A.pdf) and Szeliski 4.1.1. See Algorithm 4.1 in the textbook for pseudocode. You do not need to worry about scale invariance or keypoint orientation estimation for your baseline Harris corner detector. The original paper by Chris Harris and Mike Stephens describing their corner detector can be found [here](http://www.bmva.org/bmvc/1988/avc-88-023.pdf). We will be implementing the Harris detector using a Neural Network - HarrisNet. Our network has 5 layers (all of which you will have to implement), described briefly below:

   * **ImageGradientsLayer**  - retrieves image gradients in each direction. This layer is already implemented for you, but you’ll need to implement get_sobel_xy_parameters() in torch_layer_utils.py for it to work.
   * **ChannelProductLayer**  - returns product between channel of the previous layer Ixx, Iyy and Ixy.
   * **SecondMomentMatrixLayer** - computes Second Moment Matrix.
   * **CornerResponseLayer**  - computes the R cornerness matrix over the entire image.
   * **NMSLayer** - performs non-maxima suppression to keep only the strongest corners in local regions.
   
After passing images through the entire network, we still need to extract specific coordinates as our interest points, which is done in get_interest_points() (you will implement this) in HarrisNet.py.

In [3]:
## Verify each layer in the code, this will check if your implementation is correct or not.

## Do not modify the constructor of any layer (i.e. to take some custom arguments
## as input)


from unit_tests.harris_unit_test import (
    test_ImageGradientsLayer,
    test_ChannelProductLayer, 
    test_SecondMomentMatrixLayer, 
    test_CornerResponseLayer, 
    test_NMSLayer,
    verify
)

print('ImageGradientsLayer:', verify(test_ImageGradientsLayer))
print('ChannelProductLayer:', verify(test_ChannelProductLayer))
print('SecondMomentMatrixLayer:', verify(test_SecondMomentMatrixLayer)) #maybe wrong
print('CornerResponseLayer:', verify(test_CornerResponseLayer) )
print('NMSLayer:', verify(test_NMSLayer))


ImageGradientsLayer: "Correct"
ChannelProductLayer: "Correct"
SecondMomentMatrixLayer: "Correct"
CornerResponseLayer: "Correct"
NMSLayer: "Correct"


In [4]:
#Weilly -Solve 1.1.1 - ImageGradientsLayer
import torch
x = torch.tensor([[2, 8, 0],[2, 4, 4],[1, 5, 5]]).unsqueeze(0).unsqueeze(0).float()
kernel= torch.nn.Conv2d(1, 2, kernel_size=3, stride=1, padding=1)
kernel.weight.data[0, 0].copy_(torch.FloatTensor([[-1, 0, 1],[-2, 0, 2],[-1, 0, 1]]))
kernel.weight.data[1, 0].copy_(torch.FloatTensor([[1, 2,-1],[0, 0, 0],[1, 2, 1]]))
kernel.bias.data.zero_()
kernel(x)

tensor([[[[ 20.,  -2., -20.],
          [ 21.,   6., -21.],
          [ 14.,  10., -14.]],

         [[  8.,  14.,  12.],
          [  3.,  34.,  23.],
          [  0.,   6.,  12.]]]], grad_fn=<ThnnConv2DBackward>)

In [5]:
x = torch.tensor( [[[4, 3, 0],[0, 3, 2],[4, 2, 2]],[[2, 2, 0],[2, 1, 0],[3, 2, 1]] ]).unsqueeze(0).float()
B,C,W,H = list(x.size())
Ix2 = x[:,0,:,:]**2
Iy2 = x[:,1,:,:]**2
Ixy = x[:,0,:,:]*x[:,1,:,:]
output = torch.cat([Ix2,Iy2,Ixy],0)
print(output.size())
output = output.reshape(1,3,W,H)
print(output.size())

torch.Size([3, 3, 3])
torch.Size([1, 3, 3, 3])


In [6]:
#Weilly -Solve 1.1.2 - ChannelProductLayer
#tutorials : https://zhuanlan.zhihu.com/p/58109107
x = torch.tensor([[[4, 3, 0],[0, 3, 2],[4, 2, 2]],[[2, 2, 0],[2, 1, 0],[3, 2, 1]]]).unsqueeze(0).float()#torch.Size([1, 2, 3, 3])
B,C,W,H = list(x.size())
Ix2 = x[:,0,:,:]**2
Iy2 = x[:,1,:,:]**2
Ixy = x[:,0,:,:]*x[:,1,:,:]
output = torch.cat([Ix2,Iy2,Ixy],0)
output = output.reshape(1,C+1,W,H)
print(output,output.size())

tensor([[[[16.,  9.,  0.],
          [ 0.,  9.,  4.],
          [16.,  4.,  4.]],

         [[ 4.,  4.,  0.],
          [ 4.,  1.,  0.],
          [ 9.,  4.,  1.]],

         [[ 8.,  6.,  0.],
          [ 0.,  3.,  0.],
          [12.,  4.,  2.]]]]) torch.Size([1, 3, 3, 3])


In [7]:
#Weilly -Solve 1.1.3 - SecondMomentMatrixLaye
#https://www.jianshu.com/p/133ee1dde482
x = torch.tensor([[[[16.,  9.,  0.],[ 0.,  9.,  4.],[16.,  4.,  4.]],[[ 4.,  4.,  0.],[ 4.,  1.,  0.],[ 9.,  4.,  1.]],[[ 8.,  6.,  0.],[ 0.,  3.,  0.],[12.,  4.,  2.]]]]).float()
Ix2_Sum = (x[:,0,:,:]**2).sum()
Iy2_Sum = (x[:,1,:,:]**2).sum()
Ixy_Sum = (x[:,0,:,:]*x[:,1,:,:]).sum()
second_moment_matrix = torch.tensor([[Ix2_Sum,Ixy_Sum],[Ixy_Sum,Iy2_Sum]])
print(second_moment_matrix)

tensor([[722., 273.],
        [273., 147.]])


In [8]:
#Weilly -Solve 1.1.3.1 - get_gaussian_kernel
#https://towardsdatascience.com/implement-canny-edge-detection-from-scratch-with-pytorch-a1cccfa58bed
ksize = 3
sigma = 3

# built the 1 dimension gaussian
gaussian_1D = np.linspace(-1, 1, ksize)

# compute a grid distance from center
x, y = np.meshgrid(gaussian_1D, gaussian_1D)
distance = (x ** 2 + y ** 2) ** 0.5

# compute the 2 dimension gaussian
gaussian_2D = np.exp(-(distance - 0) ** 2 / (2 * sigma ** 2))
gaussian_2D = gaussian_2D / (2 * np.pi *sigma **2)

gaussian_filter  = gaussian_2D
gaussian_filter  = gaussian_filter/gaussian_filter.sum()

# output kernel
kernel = torch.nn.Parameter(torch.from_numpy(gaussian_filter), requires_grad=False)
kernel


Parameter containing:
tensor([[0.1070, 0.1131, 0.1070],
        [0.1131, 0.1196, 0.1131],
        [0.1070, 0.1131, 0.1070]], dtype=torch.float64)

In [9]:
#Weilly -Solve 1.1.3.2 - SecondMomentMatrixLayer 
x = torch.tensor([[[[16.,  9.,  0.],[ 0.,  9.,  4.],[16.,  4.,  4.]],[[ 4.,  4.,  0.],[ 4.,  1.,  0.],[ 9.,  4.,  1.]],[[ 8.,  6.,  0.],[ 0.,  3.,  0.],[12.,  4.,  2.]]]]).float()
B,C,W,H = list(x.size())
x1, x2, x3  = torch.split(x, 1, dim = 1)
gaussian_kernel= torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1)
gaussian_kernel.weight.data[0, 0].copy_(kernel)
gaussian_kernel.bias.data.zero_()
x1_new = gaussian_kernel(x1)
x2_new = gaussian_kernel(x2)
x3_new = gaussian_kernel(x3)
output = torch.cat([x1_new,x2_new,x3_new],0)
output=torch.reshape(output,(1,C,W,H))

In [10]:
#Weilly -Solve 1.1.4 - CornerResponseLayer 
S = torch.tensor(
    [
      [[4, 3, 0],
      [0, 3, 2],  #S_xx
      [4, 2, 2]],

      [[2, 2, 0],
      [2, 1, 0], #S_yy
      [3, 2, 1]],

      [[3, 0, 3],
      [4, 4, 1], #S_xy
      [2, 0, 1]]
    ]).unsqueeze(0).float()

B,C,W,H = list(S.size())
print(S.size())

a = S[:,0,:,:]
b = S[:,2,:,:]
c = S[:,1,:,:]
d = S[:,2,:,:]

det_matrix_A = (a*c)-(b*d)
trace_matrix = a+c

alpha = 0.05
corner_strength = det_matrix_A - alpha *(trace_matrix * trace_matrix)
corner_strength = corner_strength.view(1,W,H).unsqueeze(0)
print(corner_strength.size())

torch.Size([1, 3, 3, 3])
torch.Size([1, 1, 3, 3])


In [11]:
#Weilly -Solve 1.1.4 - NMSLayer
R = torch.tensor(
    [
      [1, 4, 4],
      [0, 1, 1],
      [2, 2, 2]
    ]).unsqueeze(0).unsqueeze(0).float()


B,C,W,H = list(R.size())
Rmax = R.max().numpy()

corner =0
output = R.clone().detach()
for h in range(W):
      for w in range(H):
            if(R[:,:,w,h] >=int(Rmax) ):
                corner = corner+1
            else :
                R[:,:,w,h]  = 0
            #if R[:,:,w,h]  > 0.01*Rmax and R[:,:,w,h] > R[:,:,w-1,h-1] and R[:,:,w,h] > R[:,:,w-1,h] and R[:,:,w,h] > R[:,:,w-1,h+1] and R[:,:,w,h] > R[:,:,w,h-1] and R[:,:,w,h] > R[:,:,w,h+1] and R[:,:,w,h] > R[:,:,w+1,h-1] and R[:,:,w,h] > R[:,:,w+1,h] and R[:,:,w,h] > R[:,:,w+1,h+1] :
               
print(R)
                    

tensor([[[[0., 4., 4.],
          [0., 0., 0.],
          [0., 0., 0.]]]])


In [12]:
image1.shape

(1024, 768, 3)

In [13]:
#Weilly -Solve 1.2 
from proj3_code.HarrisNet import ImageGradientsLayer,ChannelProductLayer,SecondMomentMatrixLayer,CornerResponseLayer,NMSLayer
B,C,W,H = list(image_input1.size())

#ImageGradientsLayer
kernel = ImageGradientsLayer()
image = kernel(image_input1)
img_Ix = image[:,0,:,:].view(W,H).detach().numpy()
img_Iy = image[:,1,:,:].view(W,H).detach().numpy()
plt.figure(figsize=(4,4)); plt.imshow((img_Ix*255).astype(np.uint8), cmap='gray');plt.title('img_Ix')
plt.figure(figsize=(4,4)); plt.imshow((img_Iy*255).astype(np.uint8), cmap='gray');plt.title('img_Iy')

#ChannelProductLayer
kernel = ChannelProductLayer()
image = kernel(image)
img_Ixx = image[:,0,:,:].view(W,H).detach().numpy()
img_Iyy = image[:,1,:,:].view(W,H).detach().numpy()
img_Ixy = image[:,2,:,:].view(W,H).detach().numpy()
plt.figure(figsize=(4,4)); plt.imshow((img_Ixx*255).astype(np.uint8), cmap='gray');plt.title('img_Ixx')
plt.figure(figsize=(4,4)); plt.imshow((img_Iyy*255).astype(np.uint8), cmap='gray');plt.title('img_Iyy')
plt.figure(figsize=(4,4)); plt.imshow((img_Ixy*255).astype(np.uint8), cmap='gray');plt.title('img_Ixy')


#SecondMomentMatrixLayer
kernel = SecondMomentMatrixLayer(ksize=3, sigma=3)
image = kernel(image)
img_Ixx_Gaussian = image[:,0,:,:].view(W,H).detach().numpy()
img_Iyy_Gaussian = image[:,1,:,:].view(W,H).detach().numpy()
img_Ixy_Gaussian = image[:,2,:,:].view(W,H).detach().numpy()
plt.figure(figsize=(4,4)); plt.imshow((img_Ixx_Gaussian*255).astype(np.uint8), cmap='gray');plt.title('img_Ixx_Gaussian')
plt.figure(figsize=(4,4)); plt.imshow((img_Iyy_Gaussian*255).astype(np.uint8), cmap='gray');plt.title('img_Iyy_Gaussian')
plt.figure(figsize=(4,4)); plt.imshow((img_Ixy_Gaussian*255).astype(np.uint8), cmap='gray');plt.title('img_Ixy_Gaussian')


#CornerResponseLayer
kernel = CornerResponseLayer(alpha=0.05)
R = kernel(image)
print(R.size())

#NMSLayer
kernel = NMSLayer()
R_nms = kernel(R)

#get feature point
R_nms_new = R_nms.view(1024,768)
Point = np.where(R_nms_new > 0)
image1_bw = rgb2gray(image1)
plt.figure(figsize=(4,4)); plt.imshow((image1*255).astype(np.uint8), cmap='gray');
plt.plot(Point[1], Point[0], '*', color='red')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

torch.Size([1, 1, 1024, 768])


<IPython.core.display.Javascript object>

In [14]:
from proj3_code.HarrisNet import get_interest_points
x1, y1, _ = get_interest_points(image_input1)
image1_bw = rgb2gray(image1)
plt.figure(figsize=(4,4)); plt.imshow((image1_bw*255).astype(np.uint8), cmap='gray');
plt.plot(y1, x1, '*', color='red')

torch.Size([1024, 768])


<IPython.core.display.Javascript object>

In [15]:
from proj3_code.HarrisNet import get_interest_points
x2, y2, _ = get_interest_points(image_input2)
image2_bw = rgb2gray(image2)
plt.figure(figsize=(4,4)); plt.imshow((image2_bw*255).astype(np.uint8), cmap='gray');
plt.plot(y2, x2, '*', color='red')

torch.Size([1016, 762])


<IPython.core.display.Javascript object>

Here we will call get_interest_points function in HarrisNet.py to detect 'interesting' points in the images. 

**IMPORTANT**
Make sure to add your code in get_interest_points function to call Harris Corner Detector.

In [16]:
from utils import show_interest_points

x1, y1, _ = get_interest_points(image_input1)
x2, y2, _ = get_interest_points(image_input2)

#x1, x2 = x1.detach().numpy(), x2.detach().numpy()
#y1, y2 = y1.detach().numpy(), y2.detach().numpy()

# Visualize the interest points
c1 = show_interest_points(image1, y1, x1)#Weilly
c2 = show_interest_points(image2, y2, x2)
plt.figure(); plt.imshow(c1)
plt.figure(); plt.imshow(c2)
print('{:d} corners in image 1, {:d} corners in image 2'.format(len(x1), len(x2)))



torch.Size([1024, 768])
torch.Size([1016, 762])


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

2870 corners in image 1, 3690 corners in image 2


# Part 2: Sift Feature Descriptor
## Create feature vectors at each interest point (Szeliski 4.1.2)

<img src="figures/SIFTNet.png" alt="drawing" width="800" title="Overview of SIFTNet pipeline"/>


You will implement a SIFT-like local feature based on the lecture materials and Szeliski 4.1.2. We will be implementing Sift using a neaural network - SIFTNet. This network has 4 layers (which you will have to implement unless specified otherwise), described briefly below:

* **ImageGradientsLayer** - computes image gradients in each direction (already implemented for you).
* **SIFTOrientationLayer** - extracts gradient information along each orientation direction. In the original SIFT, we would be trying to find the contributions of our gradients to each orientation bin. Note that we can do this by trying to find the contribution of each gradient along each orientation vector, which is the same as finding the projection of our gradients onto our orientation vectors. Recall that this can be done using dot products!
* **HistogramLayer** - creates weighted histograms over the entire image.
* **SubGridAccumulationLayer** - creates feature vectors that accumulate histograms from a region.
After passing images through the network we will have feature vectors over the entire image, but we need only want features from the specific interest point locations that we found. This will be done in get_SIFTNet_features() (you will implement this) in SIFTNet.py.

In [17]:
from proj3_code.SIFTNet import (
    angles_to_vectors_2d_pytorch,
    HistogramLayer,
    SubGridAccumulationLayer,
    SIFTOrientationLayer,
    get_sift_subgrid_coords,
    get_siftnet_features
)
from proj3_code.torch_layer_utils import ImageGradientsLayer
from unit_tests.sift_unit_test import (
    test_angles_to_vectors_2d_pytorch,
    test_HistogramLayer,
    test_ImageGradientsLayer,
    test_SubGridAccumulationLayer,
    test_SIFTOrientationLayer,
    test_get_sift_subgrid_coords,
    test_SIFTNet,
    test_get_siftnet_features
)

print('angles_to_vectors_2d_pytorch:', verify(test_angles_to_vectors_2d_pytorch))
print('HistogramLayer:', verify(test_HistogramLayer))
print('ImageGradientsLayer:', verify(test_ImageGradientsLayer))
print('SIFTOrientationLayer:', verify(test_SIFTOrientationLayer) )
print('SIFTNet:', verify(test_SIFTNet) )
print('SubGridAccumulationLayer:', verify(test_SubGridAccumulationLayer))
print('get_sift_subgrid_coords:', verify(test_get_sift_subgrid_coords) )
print('get_siftnet_features:', verify(test_get_siftnet_features))

angles_to_vectors_2d_pytorch: "Correct"
HistogramLayer: "Correct"
ImageGradientsLayer: "Correct"
SIFTOrientationLayer: "Correct"
SIFTNet: "Correct"
SubGridAccumulationLayer: "Correct"
get_sift_subgrid_coords: "Correct"
get_siftnet_features: "Correct"


In [32]:
print('HistogramLayer:', verify(test_HistogramLayer))

tensor([[[0],
         [0],
         [7],
         [1]]])
HistogramLayer: "Correct"


In [34]:
print('HistogramLayer:', verify(test_HistogramLayer))

tensor([[[ 0.2000],
         [20.0000],
         [ 0.3000],
         [30.0000]]])
HistogramLayer: "Correct"


In [19]:
#Weilly -Solve 2.1 test_angles_to_vectors_2d_pytorch
angles = torch.from_numpy(np.array([np.pi/4, np.pi, -np.pi, 0]))
gt_vectors_2d = torch.tensor([[ np.sqrt(2)/2, np.sqrt(2)/2],[-1.,  0],[-1.,  0.],[ 1.,  0.]], dtype=torch.float64)

print(angles)
print(gt_vectors_2d)

angle_vectors = torch.cat([torch.cos(angles),torch.sin(angles)],0)
angle_vectors = angle_vectors.view(2,4)
angle_vectors = angle_vectors.t()
print(angle_vectors)

angle_vectors.size()

tensor([ 0.7854,  3.1416, -3.1416,  0.0000], dtype=torch.float64)
tensor([[ 0.7071,  0.7071],
        [-1.0000,  0.0000],
        [-1.0000,  0.0000],
        [ 1.0000,  0.0000]], dtype=torch.float64)
tensor([[ 7.0711e-01,  7.0711e-01],
        [-1.0000e+00,  1.2246e-16],
        [-1.0000e+00, -1.2246e-16],
        [ 1.0000e+00,  0.0000e+00]], dtype=torch.float64)


torch.Size([4, 2])

In [20]:
#Weilly -Solve 2.2 test_HistogramLayer
x = torch.tensor([[
    [[  0.1962],
     [ 19.6157],
     [  0.2494],
     [ 24.9441]],
    
    [[  0.1111],
     [ 16.6294],
     [  0.0585],
     [ 29.4236]],
    
    [[ -0.0390],
     [  3.9018],
     [ -0.1667],
     [ 16.6671]],
    
    [[ -0.1663],
     [-11.1114],
     [ -0.2942],
     [ -5.8527]],
    
    [[ -0.1962],
     [-19.6157],
     [ -0.2494],
     [-24.9441]],
    
    [[ -0.1111],
     [-16.6294],
     [ -0.0585],
     [-29.4236]],
    
    [[  0.0390],
     [ -3.9018],
     [  0.1667],
     [-16.6671]],
    
    [[  0.1663],
     [ 11.1114],
     [  0.2942],
     [  5.8527]],
    
    [[  0.1962],
     [ 16.6294],
     [  0.2942],
     [ 16.6671]],
    
    [[  0.0390],
     [ 11.1114],
     [ -0.0585],
     [ 24.9441]]]
])
print("input x =", x.size()) #data


#get image gradient, dx, dy
cosines  = x[:,:8,:,:] # Contains image gradient projections, cos(theta) , sin(theta) ,
im_grads = x[:,8:,:,:] # Contains dx, dy
print("gradient = ",cosines.size())
print("gradient = ",cosines.size())
print("dx,dy = ",im_grads.size(),"\n")

#
new = torch.zeros(cosines.shape)
maxIdx = cosines.argmax(1) #find out max-index [dx1,dx2,dx3,dx4]
for i in range(new.shape[2]):
    for j in range(new.shape[3]):
        new[0,maxIdx[0,i,j],i,j] = 1 #find out max of mage gradient projections
        
per_px_histogram = torch.mul(torch.norm(im_grads, dim=1), new) #hist = gradient * weight
print("Max of Index = \n", maxIdx,"\n")  
print("im_grads=\n",im_grads,"\n")
print("torch.norm(im_grads, dim=1)=\n",torch.norm(im_grads, dim=1),"\n")#Norm


#ref = https://zhuanlan.zhihu.com/p/260162240
f = per_px_histogram.sum(dim=2)
print("per_px_histogram=\n",per_px_histogram)
print("f=\n",f) # histogram[bin]


input x = torch.Size([1, 10, 4, 1])
gradient =  torch.Size([1, 8, 4, 1])
gradient =  torch.Size([1, 8, 4, 1])
dx,dy =  torch.Size([1, 2, 4, 1]) 

Max of Index = 
 tensor([[[0],
         [0],
         [7],
         [1]]]) 

im_grads=
 tensor([[[[ 0.1962],
          [16.6294],
          [ 0.2942],
          [16.6671]],

         [[ 0.0390],
          [11.1114],
          [-0.0585],
          [24.9441]]]]) 

torch.norm(im_grads, dim=1)=
 tensor([[[ 0.2000],
         [20.0000],
         [ 0.3000],
         [30.0000]]]) 

per_px_histogram=
 tensor([[[[ 0.2000],
          [20.0000],
          [ 0.0000],
          [ 0.0000]],

         [[ 0.0000],
          [ 0.0000],
          [ 0.0000],
          [30.0000]],

         [[ 0.0000],
          [ 0.0000],
          [ 0.0000],
          [ 0.0000]],

         [[ 0.0000],
          [ 0.0000],
          [ 0.0000],
          [ 0.0000]],

         [[ 0.0000],
          [ 0.0000],
          [ 0.0000],
          [ 0.0000]],

         [[ 0.0000],
       

In [21]:
#Weilly -Solve 2.3 test_ImageGradientsLayer
imgrad_layer = ImageGradientsLayer()
img = load_image(f'../data/1a_notredame.jpg')
image1_bw = rgb2gray(img)
image1_bw = torch.from_numpy(image1_bw).unsqueeze(0).unsqueeze(0)

im_grads = imgrad_layer(image1_bw)
im_grads = im_grads.detach()

gt_crop = torch.tensor([[[ 0.0257, -0.0184],[ 0.0243,  0.0104]],[[ 0.0330,  0.0410],[-0.0219, -0.0026]]])
assert torch.allclose(gt_crop, im_grads[0,:,500:502,500:502], atol=1)

[b,c,h,w]=list(im_grads.size())

plt.figure()
plt.imshow(im_grads[:,0,:,:].numpy().reshape((h, w)),cmap='gray' )

plt.figure()
plt.imshow(im_grads[:,1,:,:].numpy().reshape((h, w)),cmap='gray' )

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
#Weilly -Solve 2.4 test_SIFTOrientationLayer
im_grads = torch.tensor([[[[ 0.1962],[16.6294],[ 0.2942],[16.6671]],   #dx
                          [[ 0.0390],[11.1114],[-0.0585],[24.9441]]]]) #dy

kernel = torch.nn.Conv2d(2, 8, 1, bias=None)

#weight = [cos,sin] for different angle
weights = angles_to_vectors_2d_pytorch(torch.from_numpy(np.array([np.pi / 8, 3 * np.pi / 8,5 * np.pi / 8, 7 * np.pi / 8, 9 * np.pi / 8, 11 * np.pi / 8, 13 * np.pi / 8, 15 * np.pi / 8,])).float()) # weights.shape [8, 2]
weight_param = weights.unsqueeze(2).unsqueeze(3)

#weight value
print("weights=\n",weights,"\n")
print("weight_param=\n",weight_param,"\n")

#weight.size
print("weights size=\n",weights.size(),"\n")
print("weight_param size=\n",weight_param.size(),"\n")

kernel.weight = torch.nn.Parameter(weight_param)

print("kernel(im_grads)=\n",kernel(im_grads),"\n")#torch.Size([1, 8, 4, 1])
print("torch.cat((kernel(im_grads), im_grads), 1)= \n",torch.cat((kernel(im_grads), im_grads), 1)) #torch.Size([1, 10, 4, 1])

weights=
 tensor([[ 0.9239,  0.3827],
        [ 0.3827,  0.9239],
        [-0.3827,  0.9239],
        [-0.9239,  0.3827],
        [-0.9239, -0.3827],
        [-0.3827, -0.9239],
        [ 0.3827, -0.9239],
        [ 0.9239, -0.3827]]) 

weight_param=
 tensor([[[[ 0.9239]],

         [[ 0.3827]]],


        [[[ 0.3827]],

         [[ 0.9239]]],


        [[[-0.3827]],

         [[ 0.9239]]],


        [[[-0.9239]],

         [[ 0.3827]]],


        [[[-0.9239]],

         [[-0.3827]]],


        [[[-0.3827]],

         [[-0.9239]]],


        [[[ 0.3827]],

         [[-0.9239]]],


        [[[ 0.9239]],

         [[-0.3827]]]]) 

weights size=
 torch.Size([8, 2]) 

weight_param size=
 torch.Size([8, 2, 1, 1]) 

kernel(im_grads)=
 tensor([[[[  0.1962],
          [ 19.6157],
          [  0.2494],
          [ 24.9441]],

         [[  0.1111],
          [ 16.6294],
          [  0.0585],
          [ 29.4236]],

         [[ -0.0391],
          [  3.9018],
          [ -0.1666],
          [ 16.

In [23]:
#Weilly -Solve 2.4 SIFT
img = load_image(f'../data/1a_notredame.jpg')
image1_bw = rgb2gray(img)
image1_bw = torch.from_numpy(image1_bw).unsqueeze(0).unsqueeze(0)

im_grads  = ImageGradientsLayer()(image1_bw).detach() #計算梯度
hog_datas = SIFTOrientationLayer()(im_grads) #計算梯度方向,大小
per_px_histogram = HistogramLayer()(hog_datas) #統計直方圖
#Accumulation = SubGridAccumulationLayer()(per_px_histogram)
#plt.figure(),plt.imshow(im_grads[:,0,:,:].numpy().reshape((h, w)),cmap='gray' )
#plt.figure(),plt.imshow(im_grads[:,1,:,:].numpy().reshape((h, w)),cmap='gray' )

In [24]:
Accumulation = SubGridAccumulationLayer()(per_px_histogram)

In [25]:
print("im_grads of size  = ",im_grads.size())
print("hog_datas of size = ",hog_datas.size())
print("per_px_histogram of size = ",per_px_histogram.size())
print("per_px_histogram & Accumulationof size = ",Accumulation.size())

im_grads of size  =  torch.Size([1, 2, 2048, 1536])
hog_datas of size =  torch.Size([1, 10, 2048, 1536])
per_px_histogram of size =  torch.Size([1, 8, 2048, 1536])
per_px_histogram & Accumulationof size =  torch.Size([1, 8, 2049, 1537])


In [26]:
from utils import show_interest_points

b1,c1,w1,h1=list(image_input1.size())
b2,c2,w2,h2=list(image_input1.size())

if w1>h1 : 
    square1 = h1
else: 
    square1 = w1
    
if w2>h2 : 
    square2 = h2
else: 
    square2 = w2
    
x1, y1, _ = get_interest_points(image_input1)
x2, y2, _ = get_interest_points(image_input2)

#x1, x2 = x1.detach().numpy(), x2.detach().numpy()
#y1, y2 = y1.detach().numpy(), y2.detach().numpy()

# Visualize the interest points
c1 = show_interest_points(image1, y1, x1)#Weilly[-square1:,:square1]
c2 = show_interest_points(image2, y2, x2)
plt.figure(); plt.imshow(c1)
plt.figure(); plt.imshow(c2)
#print('{:d} corners in image 1, {:d} corners in image 2'.format(len(x1), len(x2)))

torch.Size([1024, 768])
torch.Size([1016, 762])


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [27]:
w1

1024

In [28]:
#image1_features = get_siftnet_features(image_input1, x1[:938], y1[:938])
#image2_features = get_siftnet_features(image_input2, x2[:938], y2[:938])
image1_features = get_siftnet_features(image_input1, x1, y1)
image2_features = get_siftnet_features(image_input2, x2, y2)

IndexError: index 928 is out of bounds for axis 0 with size 769

## Match features (Szeliski 4.1.3)

You will implement the “ratio test” or “nearest neighbor distance ratio test” method of matching local features as described in the lecture materials and Szeliski 4.1.3. See equation 4.18 in particular. You will implement this in student_feature_matching.py. The potential matches that pass the ratio test the easiest should have a greater tendency to be correct matches–think about why.

In [ ]:
#test your feature matching implementation
from unit_tests.feature_match_test import test_feature_matching, test_compute_dists
print('compute_dists:', verify(test_compute_dists))
print('feature_matching:', verify(test_feature_matching))

In [ ]:
from student_feature_matching import match_features
matches, confidences = match_features(image1_features, image2_features, x1, y1, x2, y2)
print('{:d} matches from {:d} corners'.format(len(matches), len(x1)))

In [ ]:
matches

## Visualization

You might want to set 'num_pts_to_visualize' and 'num_pts_to_evaluate' to some constant (e.g. 100) once you start detecting hundreds of interest points, otherwise things might get too cluttered. You could also threshold based on confidence.  
  
There are two visualization functions below. You can comment out one of both of them if you prefer.

In [ ]:
from proj3_code.utils import show_correspondence_circles, show_correspondence_lines
# num_pts_to_visualize = len(matches)
num_pts_to_visualize = 17
c1 = show_correspondence_circles(image1[-square1:square1], image2[-square2:square2],
                    x1[matches[:num_pts_to_visualize, 0]], y1[matches[:num_pts_to_visualize, 0]],
                    x2[matches[:num_pts_to_visualize, 1]], y2[matches[:num_pts_to_visualize, 1]])
plt.figure(); plt.imshow(c1)
#plt.savefig('../results/vis_circles.jpg', dpi=1000)
c2 = show_correspondence_lines(image1[-square1:square1], image2[-square2:square2],
                    x1[matches[:num_pts_to_visualize, 0]], y1[matches[:num_pts_to_visualize, 0]],
                    x2[matches[:num_pts_to_visualize, 1]], y2[matches[:num_pts_to_visualize, 1]])
plt.figure(); plt.imshow(c2)
#plt.savefig('../results/vis_lines.jpg', dpi=1000)

Comment out the function below if you are not testing on the Notre Dame, Episcopal Gaudi, and Mount Rushmore image pairs--this evaluation function will only work for those which have ground truth available.  
  
You can use `annotate_correspondences/collect_ground_truth_corr.py` to build the ground truth for other image pairs if you want, but it's very tedious. It would be a great service to the class for future years, though!

In [ ]:
from proj3_code.utils import evaluate_correspondence
# num_pts_to_evaluate = len(matches)
num_pts_to_evaluate = 100
_, c = evaluate_correspondence(image1, image2, eval_file, scale_factor,
                        x1[matches[:num_pts_to_evaluate, 0]], y1[matches[:num_pts_to_evaluate, 0]],
                        x2[matches[:num_pts_to_evaluate, 1]], y2[matches[:num_pts_to_evaluate, 1]])
plt.figure(); plt.imshow(c)
plt.savefig('../results/eval.jpg', dpi=1000)